In [1]:
from astropy.coordinates import SkyCoord
from astropy.coordinates.name_resolve import NameResolveError
from astropy.table import Table, vstack
from astropy.time import Time

from pathlib import Path

import pickle

In [2]:
DATA_PATH = Path("//stem-linux-homes/OSL-Telescope/data/users/Pipeline/PIRATE/")

In [3]:
try: 
    with open('data/processed_dates.pickle', 'rb') as processed_dates_file:
        processed_dates = pickle.load(processed_dates_file)
except FileNotFoundError:
    processed_dates = []

In [4]:
try: 
    with open('data/coords.pickle', 'rb') as coords_file:
        coords = pickle.load(coords_file)
except FileNotFoundError:
    coords = {}

To do:
- Take account of flags
- Calibration

In [5]:
new_dates = [p for p in DATA_PATH.glob('202?_??_??') if p not in processed_dates]
obs_tables = {}
for date in new_dates:
    for catalogue in date.glob('Catalogues/*_anm83_*.cat'):
        obs_meta = catalogue.stem.split('_')
        obs_meta = {
            'name': obs_meta[5],
            'band': obs_meta[7][0],
            'exposure': obs_meta[7][1:],
            'timestamp': Time(
                dict(zip(
                    ['year', 'month', 'day', 'hour', 'minute', 'second'],
                    map(int, obs_meta[9:15])
                )),
                format='ymdhms',
            ).jd,
        }
        if obs_meta['name'] not in coords:
            try:
                coords[obs_meta['name']] = SkyCoord.from_name(obs_meta['name'], parse=True)
            except NameResolveError:
                continue
        
        table = Table.read(catalogue, format='ascii.sextractor')
        table.rename_column('ALPHA_J2000', 'RA')
        table.rename_column('DELTA_J2000', 'Dec')
        
        table['separation'] = SkyCoord.guess_from_table(table).separation(coords[obs_meta['name']])
        table.sort('separation')
        
        for key, val in obs_meta.items():
            table[key] = val
        
        if obs_meta['name'] not in obs_tables:
            try:
                obs_tables[obs_meta['name']] = Table.read(f"data/{obs_meta['name']}.ecsv")
            except FileNotFoundError:
                obs_tables[obs_meta['name']] = table[0]
                continue
        obs_tables[obs_meta['name']] = vstack([obs_tables[obs_meta['name']], table[0]])
        
    processed_dates.append(date)

In [6]:
for name, table in obs_tables.items():
    table.write(f"data/{name}.ecsv", overwrite=True)

In [7]:
with open('data/processed_dates.pickle', 'wb') as processed_dates_file:
    pickle.dump(processed_dates, processed_dates_file)

In [8]:
with open('data/coords.pickle', 'wb') as coords_file:
    pickle.dump(coords, coords_file)